In [112]:
import requests
import os
import json
import pandas as pd
from datetime import date

In [58]:
os.environ["BEARER_TOKEN"] = 'AAAAAAAAAAAAAAAAAAAAALnNPwEAAAAARPxO2h0miAqxDB4KKOzzlbdYjMQ%3DKxlk070Baka5kUmI8FrWb7Yr0vFAle7VKEIJ7yQ2r6Ks9N385i'

## functions

In [124]:
# === user list request

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    token = os.environ["BEARER_TOKEN"]
    r.headers["Authorization"] = f"Bearer {token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def get_list(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def users_request():
    list_req_url = "https://api.twitter.com/1.1/lists/members.json"
    parlListId = 864088912087715840
    query_params = {'list_id': '{}'.format(parlListId), 'count': '5000'}
    json_response = get_list(list_req_url, query_params)
    return json_response

def id_list():
    json_response = users_request()
    # extract the user IDs
    members = pd.DataFrame.from_dict(json_response['users'])
    user_ids = list(members['id'])
    return user_ids

def user_list():
    json_response = users_request()
    members = pd.DataFrame.from_dict(json_response['users'])
    mem_list = members[['id','name','screen_name']]
    return mem_list

# === user information requests

def create_headers():
    """
    create the request headers, includes the token
    """
    headers = {"Authorization": "Bearer {}".format(os.environ.get("BEARER_TOKEN"))}
    return headers

def connect_to_endpoint(url, headers):
    """
    connect to the API and return the json response
    """
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def main(search_url):
    headers = create_headers()
    json_response = connect_to_endpoint(search_url, headers)
    #return json.dumps(json_response, indent=4, sort_keys=True)
    return json_response

In [60]:
os.environ["BEARER_TOKEN"] = 'AAAAAAAAAAAAAAAAAAAAALnNPwEAAAAARPxO2h0miAqxDB4KKOzzlbdYjMQ%3DKxlk070Baka5kUmI8FrWb7Yr0vFAle7VKEIJ7yQ2r6Ks9N385i'

#### return the list of users

In [61]:
# get the current list of MPs
user_id_list = id_list()
print(len(user_id_list))

321


## get all the tweets

In [62]:
# list to hold all of the json responses
aggregate_data = []

In [75]:
for user in user_id_list:
    # define which fields will be returned
    tweet_fields = 'created_at,author_id,conversation_id,entities,in_reply_to_user_id,referenced_tweets,text,public_metrics'
    user_fields = 'username,public_metrics,verified,url,id'
    max_results = '30'
    
    print(user)
    
    # the user request url
    search_url = "https://api.twitter.com/2/users/{}/tweets?tweet.fields={}&user.fields={}&max_results={}".format(user,tweet_fields,user_fields,max_results)
    # print(search_url)
    response = main(search_url)
    
    aggregate_data.append(response)

1223994797834297350
200
1204044181037101057
200
1198299741060239360
200
1192858611879428097
200
1187132490684784642
200
1173429508034220032
200
1172345859905871872
200
1170770038208565248
200
1166717268975333376


KeyboardInterrupt: 

In [66]:
# check that we have all the MPs
print(len(aggregate_data))

44


In [67]:
print(aggregate_data[0]) # one tweet

{'data': [{'id': '1319085046934929410', 'created_at': '2020-10-22T01:15:57.000Z', 'public_metrics': {'retweet_count': 23, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'conversation_id': '1319085046934929410', 'referenced_tweets': [{'type': 'retweeted', 'id': '1318947998857109504'}], 'author_id': '1223994797834297350', 'text': 'RT @BlocQuebecois: Depuis un an, 32 députés du #BlocQc portent fièrement la voix du Québec à Ottawa. Grâce à vous, nous avons le privilège…'}, {'id': '1318664109160878087', 'created_at': '2020-10-20T21:23:17.000Z', 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'conversation_id': '1318664109160878087', 'author_id': '1223994797834297350', 'text': 'Attention! Attention! La date butoir est repoussée au vendredi 23 octobre, 15 h pour le programme Nouveaux Horizons aux aînés 2020 #polcan https://t.co/mLEGV4eB6F'}, {'id': '1317208407133257728', 'created_at': '2020-10-16T20:58:51.000Z', 'public_metrics': {'retweet_c

### upload the list of users to the database as json

In [125]:
user_list = user_list() # get dataframe
user_list = user_list.to_json() # make it json
user_list = json.loads(user_list)
user_list['date'] = str(date.today()) # add the date of collection

print(user_list)

200
                      id                    name      screen_name
0    1223994797834297350      Charbonneau Louise  CharbonneauLou5
1    1204044181037101057           Gerald Soroka   GeraldSorokaMP
2    1198299741060239360             kyleseeback      kyleseeback
3    1192858611879428097       Mike Kelloway, MP     mikekelloway
4    1187132490684784642           Louise Chabot   LouiseChabotBQ
..                   ...                     ...              ...
316             16014404       Stéphane Bergeron        sbergeron
317             15810950       Matt Jeneroux, MP         jeneroux
318             14538949  Michelle Rempel Garner   MichelleRempel
319             14260960          Justin Trudeau    JustinTrudeau
320              3358671              Ken Hardie        KenHardie

[321 rows x 3 columns]
{'id': {'0': 1223994797834297350, '1': 1204044181037101057, '2': 1198299741060239360, '3': 1192858611879428097, '4': 1187132490684784642, '5': 1173429508034220032, '6': 11723458599

#### Single Tweet Request for Testing

In [73]:
tweet_fields = 'created_at,author_id,conversation_id,entities,in_reply_to_user_id,referenced_tweets,text,public_metrics'
# 'id,created_at,author_id,created_at,conversation_id,lang,entities,in_reply_to_user_id,referenced_tweets,text,public_metrics'  
user_fields = 'username,public_metrics,verified,url,id'
expansions = 'author_id'
max_results = '30'

# Michelle Rempel Garner
single_user = 256069692

# the user request url
single_url = "https://api.twitter.com/2/users/{}/tweets?expansions={}&tweet.fields={}&user.fields={}&max_results={}".format(single_user,expansions,tweet_fields,user_fields,max_results)

print(single_url)

single_response = main(single_url)

https://api.twitter.com/2/users/256069692/tweets?expansions=author_id&tweet.fields=created_at,author_id,conversation_id,entities,in_reply_to_user_id,referenced_tweets,text,public_metrics&user.fields=username,public_metrics,verified,url,id&max_results=30
200


In [77]:
print(single_response)
print(type(single_response))

{'data': [{'id': '1421983432830115843', 'created_at': '2021-08-01T23:57:24.000Z', 'entities': {'mentions': [{'start': 0, 'end': 9, 'username': 'K_elly_B', 'id': '378826663'}], 'annotations': [{'start': 44, 'end': 50, 'probability': 0.9746, 'type': 'Person', 'normalized_text': 'Trudeau'}]}, 'conversation_id': '1421982750702145541', 'in_reply_to_user_id': '378826663', 'text': '@K_elly_B I think to highlight that this is Trudeau’s call and his alone.\n\nOnce campaign starts, messaging will shift gears.', 'referenced_tweets': [{'type': 'replied_to', 'id': '1421982750702145541'}], 'public_metrics': {'retweet_count': 0, 'reply_count': 1, 'like_count': 1, 'quote_count': 0}, 'author_id': '256069692'}, {'id': '1421949439615193093', 'created_at': '2021-08-01T21:42:19.000Z', 'conversation_id': '1421949439615193093', 'text': 'It’s not a fall election if E-Day is before September 21st. \n\nPlease adjust your headlines accordingly.\n\nThanks.', 'public_metrics': {'retweet_count': 2, 'reply_count': 2

In [82]:
print(single_response['data'][0]) # each value in the list is a tweet - loop through all 30 to take the goods

{'id': '1421983432830115843', 'created_at': '2021-08-01T23:57:24.000Z', 'entities': {'mentions': [{'start': 0, 'end': 9, 'username': 'K_elly_B', 'id': '378826663'}], 'annotations': [{'start': 44, 'end': 50, 'probability': 0.9746, 'type': 'Person', 'normalized_text': 'Trudeau'}]}, 'conversation_id': '1421982750702145541', 'in_reply_to_user_id': '378826663', 'text': '@K_elly_B I think to highlight that this is Trudeau’s call and his alone.\n\nOnce campaign starts, messaging will shift gears.', 'referenced_tweets': [{'type': 'replied_to', 'id': '1421982750702145541'}], 'public_metrics': {'retweet_count': 0, 'reply_count': 1, 'like_count': 1, 'quote_count': 0}, 'author_id': '256069692'}


## ignore this ending part

In [87]:
first_tweet = single_response['data'][0]

first_tweet_id = int(first_tweet['author_id'])

author = user_list.loc[user_list['id'] == first_tweet_id]

In [92]:
print(author)

            id                name     screen_name
216  256069692  Alistair MacGregor  AMacGregor4CML


In [94]:
single_response_with_auth = single_response

single_response_with_auth['author_name'] = author['name']